# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sloyka.src.geocoder.geocoder import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-24 18:51:04,445 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


#### Загрузка данных

In [2]:
import pandas as pd

s_data = {
        "text": [
            "Рубинштейна 25 дворовую территорию уберите, где работники?", "На биржевой 15 снова шумят!!"
        ]
    }
df = pd.DataFrame(s_data)

In [3]:
df = pd.read_csv('/Users/test/Documents/code/sloyka/sloyka/sample_data/vnukovomos.csv', sep=';')

In [4]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name
0,2024-06-13 14:34:45,837,Жителей района Внуково приглашаем принять учас...,56.0,1,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
1,2024-06-13 14:27:37,836,Приглашаем жителей с их питомцами на площадку ...,40.0,1,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
2,2024-06-10 10:59:38,835,Алкоголь – это один из самых распространенных ...,78.0,0,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
3,2024-06-07 15:19:55,834,"Приглашаем ВСЕХ на фестиваль ""Лето в Москве. В...",117.0,6,0.0,post,https://go.mos.ru/#organizators,NaN,NaN,NaN,Управа района Внуково города Москвы
4,2024-06-07 15:08:38,833,07.06.2024 года глава Совета депутатов района ...,233.0,16,1.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
...,...,...,...,...,...,...,...,...,...,...,...,...
457,2022-04-24 16:03:16,200,Уважаемые депутаты. Прошу обратить ваше вниман...,NaN,0,NaN,comment,NaN,711231899.0,191.0,NaN,Управа района Внуково города Москвы
458,2022-04-26 08:43:40,201,[id711231899|Виктория Ii] \nДобрый день! По эт...,NaN,0,NaN,reply,NaN,-179182571.0,191.0,200.0,Управа района Внуково города Москвы
459,2022-04-26 08:47:36,202,"Спасибо за неравнодушное отношение, хотелось б...",NaN,0,NaN,reply,NaN,711231899.0,191.0,200.0,Управа района Внуково города Москвы
460,2022-05-16 13:45:58,210,[id711231899|Виктория Ii] Спасибо за ожидание....,NaN,0,NaN,reply,NaN,539274646.0,191.0,200.0,Управа района Внуково города Москвы


In [5]:
osm_id = 181288
geocoder = Geocoder(osm_id=osm_id, city_tags={'place':['town']})


2024-06-24 18:51:05,654 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


Processing category place: 100%|██████████| 1/1 [00:00<00:00, 22.50it/s]
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: De

In [6]:
result = geocoder.run(df=df.iloc[:50])
display(result)

100%|██████████| 50/50 [00:01<00:00, 25.56it/s]

06-24 18:51 | INFO     | get_stem started


06-24 18:51 | WARNING  | Error processing row with street 'внуково' and toponym 'None'
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'внуково' and toponym 'None'
06-24 18:51 | WARNING  | Error processing row with street 'славы' and toponym 'None'
06-24 18:51 | WARNING  | Error processing row with street 'славы' and toponym 'None'
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'внуково' and toponym 'None'
06-24 18:51 | WARNING  | Error processing row with street 'None' and toponym 'None
06-24 18:51 | WARNING  | Error processing row with street 'изварино' an

100%|██████████| 7/7 [00:02<00:00,  2.56it/s]


,text,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,...,id,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name
0,Жителей района Внуково приглашаем принять учас...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,837,56.0,1,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
1,Приглашаем жителей с их питомцами на площадку ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,836,40.0,1,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
2,Алкоголь – это один из самых распространенных ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,835,78.0,0,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
3,"Приглашаем ВСЕХ на фестиваль ""Лето в Москве. В...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,834,117.0,6,0.0,post,https://go.mos.ru/#organizators,NaN,NaN,NaN,Управа района Внуково города Москвы
4,07.06.2024 года глава Совета депутатов района ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,833,233.0,16,1.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
5,Лето в Москве! Все на улицу! Ждём в парке Алле...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,823,298.0,13,0.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
6,"Сегодня, в День защиты детей стартовал городск...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,822,180.0,7,1.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
7,Время встречать лето в Москве!Спортивные танцы...,изваринская,,None,Изваринская улица Внуково Россия,[Изваринская улица Внуково Россия],Изваринская улица Внуково Россия,Изваринская улица,"Изваринская улица, район Внуково, Москва, Цент...",POINT (37.25375 55.61252),...,821,105.0,1,1.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
8,"Приглашаем на открытие фестивальной площадки ""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,811,664.0,7,2.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы
9,1 июня стартует общегородской проект «Лето в М...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,810,272.0,10,3.0,post,NaN,NaN,NaN,NaN,Управа района Внуково города Москвы


In [ ]:
result

In [7]:
result.iloc[:,:].explore()